In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, remove = ('headers', 'footers'))
targets_train = twenty_train['target']

twenty_test = fetch_20newsgroups(subset = 'test', remove = ('headers', 'footers'))
targets_test = twenty_test['target']

groups = np.arange(20)
priori = np.mean(targets_train == groups.reshape(-1, 1), axis = 1)

with open('train_data.txt', 'rt') as file: train_data = file.read().splitlines()
with open('test_data.txt', 'rt') as file: test_data = file.read().splitlines() 

In [8]:
def attempt(smoothing, max_weight, **kwargs):
    cv = CountVectorizer(**kwargs, stop_words = 'english')

    fitted = cv.fit_transform(train_data)
    #f2 = fitted.toarray()

    probabilities = np.zeros((20, fitted.shape[1]))
    
    #df = np.count_nonzero(f2, axis = 0) / targets_train.size
    #plt.hist(df * 100, bins = 80)
    #plt.yscale('log')
    #plt.xlabel('%')
    #plt.ylabel('Repeticiones')

    for group in groups:
        row = fitted[targets_train == group].sum(axis = 0) + smoothing
        row[0, np.array(row == row.max())[0]] *= max_weight
        probabilities[group] = row / row.sum()
        
    #np.delete(f2)
    
    fitted_test = cv.transform(test_data)#.toarray().T
        
    probs_test = fitted_test.dot(np.log(probabilities).T) + np.log(priori)
    pred_test = np.argmax(probs_test, axis = 1)

    acc_test = np.mean(pred_test == targets_test)
    
    return {'Test': (acc_test * 100).round(2)}

In [ ]:
d = np.arange(.01, 0.3, .005)
s = np.arange(1e-9, .11, .01)

for df in d:
    for smooth in s:
        print(df, smooth)
        print(attempt(min_df = 1, max_df = df, smoothing = smooth, max_weight = 2.5))

0.01 1e-09
{'Test': 70.22}
0.01 0.010000001
{'Test': 72.42}
0.01 0.020000001
{'Test': 72.38}
0.01 0.030000001
{'Test': 72.48}
0.01 0.040000001
{'Test': 72.64}
0.01 0.050000001
{'Test': 72.62}
0.01 0.060000001
{'Test': 72.73}
0.01 0.070000001
{'Test': 72.82}
0.01 0.080000001
{'Test': 72.89}
0.01 0.090000001
{'Test': 73.04}
0.01 0.100000001
{'Test': 73.01}
0.015 1e-09
{'Test': 71.0}
0.015 0.010000001
{'Test': 73.81}
0.015 0.020000001
{'Test': 74.03}
0.015 0.030000001
{'Test': 74.2}
0.015 0.040000001
{'Test': 74.48}
0.015 0.050000001
{'Test': 74.67}
0.015 0.060000001
{'Test': 74.69}
0.015 0.070000001
{'Test': 74.79}
0.015 0.080000001
{'Test': 74.76}
0.015 0.090000001
{'Test': 74.76}
0.015 0.100000001
{'Test': 74.79}
0.019999999999999997 1e-09
{'Test': 71.51}
0.019999999999999997 0.010000001
{'Test': 74.43}
0.019999999999999997 0.020000001
{'Test': 74.75}
0.019999999999999997 0.030000001
{'Test': 74.93}
0.019999999999999997 0.040000001
{'Test': 75.07}
0.019999999999999997 0.050000001
{'Tes

In [ ]:
sens_tot = []
especif_tot = []
vp_tot = []
vn_tot = []

for i in range(20):
    targets_ok = targets_test == i
    pred_ok = pred == i
    VP = np.sum(targets_ok & pred_ok)
    VN = np.sum(~targets_ok & ~pred_ok)
    FP = np.sum(~targets_ok & pred_ok)
    FN = np.sum(targets_ok & ~pred_ok)
    
    sens_tot.append(VP / (VP + FN))
    especif_tot.append(VN / (VN + FP))
    vp_tot.append(VP / (VP + FP))
    vn_tot.append(VN / (VN + FN))
    
print(f'Sensibilidad: {np.round(np.mean(sens_tot) * 100, 2)}%')

# No tiene sentido, porque son 20 clases. Obvio que va a dar alto.
print(f'Especificidad: {np.round(np.mean(especif_tot) * 100, 2)}%') 

print(f'Valor predictivo positivo: {np.round(np.mean(vp_tot) * 100, 2)}%')

# No tiene sentido, porque son 20 clases. Obvio que va a dar alto.
print(f'Valor predictivo negativo: {np.round(np.mean(vn_tot) * 100, 2)}%')

## Preguntas
- $\textbf{Parte 1}$:
    - ¿Va bien 79.3%?
    - ¿Qué más podemos hacer para subirlo?
    - ¿Está bien usar accuracy?
    - ¿Para las otras métricas está bien sacar el promedio?
    
- $\textbf{Parte 2}$:
    - ¿Podemos sacar el priori si da peores resultados?
    - ¿Está bien elegida la sensibilidad como métrica?
    - ¿Está bien reemplazar a los NaN por la media según la clase?    

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha = .05)

cv = CountVectorizer()

mnb.fit(cv.fit_transform(train_data), targets_train)

mnb.score(cv.transform(test_data), targets_test)
